<h2>Team Members</h2>
<ul>
<li>Sohad Hossam Eldin 1190019</li>
<li>Bassant Hisham Mohamed 1190018</li>
<li>Yasmin Hashem Niazy 4200013</li>
<li>Mary Ashraf 1190322</li>
</ul>

In [48]:



%pip install pyarabic 
%pip install tkseem


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [49]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [50]:
%pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [51]:
%pip install diacritization-evaluation

Note: you may need to restart the kernel to use updated packages.


<h4>Imports</h4>

In [52]:
import re
from pyarabic.araby import strip_tashkeel
import nltk 
from nltk import word_tokenize
import tkseem 
from diacritization_evaluation import util

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

<h4>Reading the dataset</h4>

In [53]:
dataset_txt = open(r"train.txt", "r", encoding='utf-8').read()
print(type(dataset_txt))
#print(dataset_txt)

<class 'str'>


<h4>Cleaning the dataset</h4>

In [54]:
# do we need to remove [ :ص]
chars_to_remove = r"\(\s*[a-zA-Z0-9_-]+\s*/\s*[a-zA-Z0-9_-]+\s*\)|[a-zA-Z0-9_-]+" 
dataset_cleaned = re.sub(chars_to_remove, "", dataset_txt)
#print(dataset_cleaned)

<h4>Segmenting dataset into sentences</h4>

In [55]:
# r'(\([^)]*\))' should we remove the brackets or not
dataset_sentences = re.split(r"\s*\.\s*|\n|\s*،\s*|\s*:\s*|\s*[()]\s*|\s*؛\s*|\s*؟\s*|\s*!\s*|\s*\"\s*",dataset_cleaned)
print(type(dataset_sentences))
print(dataset_sentences[0:30])

<class 'list'>
['قَوْلُهُ', '', 'أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ', 'قَالَ الزَّرْكَشِيُّ', 'ابْنُ عَرَفَةَ', 'قَوْلُهُ', 'بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً', 'كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ', 'ابْنُ عَرَفَةَ', 'قَوْلُ ابْنِ شَاسٍ', 'أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ', 'وَسِحْرٍ', 'مُحَمَّدٌ', 'قَوْلُ مَالِكٍ وَأَصْحَابِهِ أَنَّ السَّاحِرَ كَافِرٌ بِاَللَّهِ تَعَالَى قَالَ مَالِكٌ', 'هُوَ كَالزِّنْدِيقِ إذَا عَمِلَ السِّحْرَ بِنَفْسِهِ قُتِلَ وَلَمْ يُسْتَتَبْ', '', 'قَوْلُهُ لِعَدَمِ مَا تَتَعَلَّقُ إلَخْ', 'أَيْ الْوَصِيَّةُ', 'قَوْلُهُ مَا مَرَّ', 'أَيْ قُبَيْلَ قَوْلِ الْمَتْنِ لَغَتْ وَلَوْ اقْتَصَرَ عَلَى أَوْصَيْت لَهُ بِشَاةٍ أَوْ أَعْطُوهُ شَاةً وَلَا غَنَمَ لَهُ عِنْدَ الْمَوْتِ هَلْ تَبْطُلُ الْوَصِيَّةُ أَوْ يُشْتَرَى لَهُ شَاةٌ وَيُؤْخَذُ مِنْ قَوْلِهِ الْآتِي كَمَا ل

<h4>Removing Tashkeel</h4>

In [56]:
dataset_without_tashkeel = strip_tashkeel(dataset_cleaned)
##print(dataset_without_tashkeel)

<h4>Tokenizing the sentence</h4>

In [57]:
tokenized_sentences = []
for sentence in dataset_sentences:
    tokens = word_tokenize(sentence, language="arabic", preserve_line=True)
    if(len(tokens)) :
        tokens.insert(0, "<s>")
        tokens.append("</s>")
        tokenized_sentences.append(tokens)

In [61]:
print(type(tokenized_sentences))
print(tokenized_sentences[0:10])
#print(tokenized_sentences[1][1])
yasmin ='الْأَوَّلُ'
text, inputs, diacritics = util.extract_haraqat(tokenized_sentences[0][1])
print(text)
print(inputs)
print(diacritics)


<class 'list'>
[['<s>', 'قَوْلُهُ', '</s>'], ['<s>', 'أَوْ', 'قَطَعَ', 'الْأَوَّلُ', 'يَدَهُ', 'إلَخْ', '</s>'], ['<s>', 'قَالَ', 'الزَّرْكَشِيُّ', '</s>'], ['<s>', 'ابْنُ', 'عَرَفَةَ', '</s>'], ['<s>', 'قَوْلُهُ', '</s>'], ['<s>', 'بِلَفْظٍ', 'يَقْتَضِيه', 'كَإِنْكَارِ', 'غَيْرِ', 'حَدِيثٍ', 'بِالْإِسْلَامِ', 'وُجُوبَ', 'مَا', 'عُلِمَ', 'وُجُوبُهُ', 'مِنْ', 'الدِّينِ', 'ضَرُورَةً', '</s>'], ['<s>', 'كَإِلْقَاءِ', 'مُصْحَفٍ', 'بِقَذَرٍ', 'وَشَدِّ', 'زُنَّارٍ', '</s>'], ['<s>', 'ابْنُ', 'عَرَفَةَ', '</s>'], ['<s>', 'قَوْلُ', 'ابْنِ', 'شَاسٍ', '</s>'], ['<s>', 'أَوْ', 'بِفِعْلٍ', 'يَتَضَمَّنُهُ', 'هُوَ', 'كَلُبْسِ', 'الزُّنَّارِ', 'وَإِلْقَاءِ', 'الْمُصْحَفِ', 'فِي', 'صَرِيحِ', 'النَّجَاسَةِ', 'وَالسُّجُودِ', 'لِلصَّنَمِ', 'وَنَحْوِ', 'ذَلِكَ', '</s>']]
قَوْلُهُ
['ق', 'و', 'ل', 'ه']
['َ', 'ْ', 'ُ', 'ُ']


## tokenizing the words

In [68]:
x_train_letters = []
y_train_letters = []
for sentence in tokenized_sentences:
    sentence_without_s=sentence[1:-1]

    x_train_letters.insert(0, "<s>")
    y_train_letters.insert(0, "")
    for word in sentence_without_s:
        text, inputs, diacritics =util.extract_haraqat(word)
        x_train_letters.append(inputs)
        y_train_letters.append(diacritics)

    
    x_train_letters.append("</s>")
    y_train_letters.append("")



KeyboardInterrupt: 

In [ ]:
print(type(x_train_letters))
print(x_train_letters[2])

#ouh lala , tokinze is dividing the tashkelat as indpendent letters as well , what does that tell us tho 

<class 'list'>
['<s>', 'أ', 'َ', 'و', 'ْ', 'ق', 'َ', 'ط', 'َ', 'ع', 'َ', 'ا', 'ل', 'ْ', 'أ', 'َ', 'و', 'ّ', 'َ', 'ل', 'ُ', 'ي', 'َ', 'د', 'َ', 'ه', 'ُ', 'إ', 'ل', 'َ', 'خ', 'ْ', '</s>']
